In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("/content/twitter-suicidal_data.csv")

In [3]:
data.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [4]:
data["intention"].value_counts()

0    5121
1    3998
Name: intention, dtype: int64

In [5]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git



  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-5m5xkwgy
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-5m5xkwgy
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.0.5-cp36-none-any.whl size=4517 sha256=930d1f028c44c2cecdbf43523f63db400f640d2fbac3d2e0064fe12d3d7a30cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-xyfn0we0/wheels/a8/18/22/90afa4bd43247fb9a75b710a4a3fcd94966c022ce9e3c7d0a6
Successfully built preprocess-kgptalkie


In [6]:
import preprocess_kgptalkie as ps

In [7]:
import re

In [8]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [9]:
data['tweet']=data['tweet'].apply(lambda x:get_clean(x))

In [10]:
data.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import  train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [12]:
tfidf=TfidfVectorizer(max_features=20000,ngram_range=(1,3),analyzer='char')

In [13]:
x=tfidf.fit_transform(data['tweet'])
y=data['intention']

In [14]:
x.shape

(9119, 10440)

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [16]:
clf=LinearSVC()
clf.fit(x_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [17]:
y_pred=clf.predict(x_test)

In [18]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1060
           1       0.91      0.91      0.91       764

    accuracy                           0.93      1824
   macro avg       0.92      0.92      0.92      1824
weighted avg       0.93      0.93      0.93      1824



In [19]:
x='today i am so happy'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([0])

In [20]:
x='today i am so happy. but i will die'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([1])